In [2]:
import pandas as pd

In [18]:
df_s1 = pd.read_csv("../ABtest/editdata/scenario1_clicklogs.csv")
df_s1

,_id,button,username,timestamp,duration_sec,total_duration_sec,scenario
0,67fc68852560c173d7061929,StartButton - scenario 1,구두자,2025-04-14 01:44:37.025000+00:00,0.000,66.845,scenario 1
1,67fc68922560c173d706192b,공기밥 추가,구두자,2025-04-14 01:44:50.878000+00:00,13.853,66.845,scenario 1
2,67fc689c2560c173d706192d,광운 김밥,구두자,2025-04-14 01:45:00.017000+00:00,9.139,66.845,scenario 1
3,67fc68a12560c173d706192f,결제 버튼,구두자,2025-04-14 01:45:05.065000+00:00,5.048,66.845,scenario 1
4,67fc68c32560c173d7061931,no,구두자,2025-04-14 01:45:39.594000+00:00,34.529,66.845,scenario 1
5,67fc68c72560c173d7061933,결제 버튼,구두자,2025-04-14 01:45:43.870000+00:00,4.276,66.845,scenario 1
6,67fc63da2560c173d70618eb,StartButton - scenario 1,박정현,2025-04-14 01:24:42.990000+00:00,0.000,26.836,scenario 1
7,67fc63df2560c173d70618ed,돈까스,박정현,2025-04-14 01:24:47.983000+00:00,4.993,26.836,scenario 1
8,67fc63e12560c173d70618ef,돈까스,박정현,2025-04-14 01:24:49.628000+00:00,1.645,26.836,scenario 1
9,67fc63e82560c173d70618f1,결제 버튼,박정현,2025-04-14 01:24:56.613000+00:00,6.985,26.836,scenario 1


In [19]:
df_s1["timestamp"] = pd.to_datetime(df_s1["timestamp"])
df_s1 = df_s1.sort_values(by=["username", "timestamp"])

# 사용자별로 처리
def get_first_response_and_time(user_df):
    user_df = user_df.reset_index(drop=True)
    for i in range(len(user_df) - 1):
        curr_btn = user_df.loc[i, "button"]
        next_btn = user_df.loc[i + 1, "button"]
        
        if curr_btn == "결제 버튼" and next_btn in ["yes", "no"]:
            time_diff = (user_df.loc[i + 1, "timestamp"] - user_df.loc[i, "timestamp"]).total_seconds()
            return pd.Series([next_btn, time_diff])
    return pd.Series(["응답 없음", None])

# 그룹별 적용
results = df.groupby("username").apply(get_first_response_and_time).reset_index()
results.columns = ["username", "결제 응답", "응답까지 걸린 시간 (초)"]

# yes / no만 필터링
filtered = results[results["결제 응답"].isin(["yes", "no"])]

# 응답별 count와 평균 시간 계산
summary_s1 = (
    filtered.groupby("결제 응답")["응답까지 걸린 시간 (초)"]
    .agg(응답수="count", 평균시간="mean")
    .reset_index()
)

# 시나리오 레이블 추가
summary_s1["시나리오"] = "scenario 1"

# 결과 출력
summary_s1


C:\Users\82103\AppData\Local\Temp\ipykernel_13068\681971341.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results = df.groupby("username").apply(get_first_response_and_time).reset_index()


,결제 응답,응답수,평균시간,시나리오
0,no,5,20.1934,scenario 1
1,yes,5,24.5778,scenario 1


In [23]:
df_s2 = pd.read_csv("../ABtest/editdata/scenario2_clicklogs.csv")
df_s2

,_id,button,username,timestamp,duration_sec,total_duration_sec,scenario
0,67fc6f532560c173d706196f,StartButton - scenario 2,김인복,2025-04-14 02:13:39.068000+00:00,0.000,26.568,scenario 2
1,67fc6f5d2560c173d7061971,된장찌개,김인복,2025-04-14 02:13:49.122000+00:00,10.054,26.568,scenario 2
2,67fc6f632560c173d7061973,결제 버튼,김인복,2025-04-14 02:13:55.394000+00:00,6.272,26.568,scenario 2
3,67fc6f6a2560c173d7061975,no,김인복,2025-04-14 02:14:02.001000+00:00,6.607,26.568,scenario 2
4,67fc6f6d2560c173d7061977,결제 버튼,김인복,2025-04-14 02:14:05.636000+00:00,3.635,26.568,scenario 2
5,67fc6e772560c173d7061965,StartButton - scenario 2,서정열,2025-04-14 02:09:59.248000+00:00,0.000,39.006,scenario 2
6,67fc6e892560c173d7061967,순두부찌개,서정열,2025-04-14 02:10:17.241000+00:00,17.993,39.006,scenario 2
7,67fc6e912560c173d7061969,결제 버튼,서정열,2025-04-14 02:10:25.761000+00:00,8.520,39.006,scenario 2
8,67fc6e9a2560c173d706196b,no,서정열,2025-04-14 02:10:34.527000+00:00,8.766,39.006,scenario 2
9,67fc6e9e2560c173d706196d,결제 버튼,서정열,2025-04-14 02:10:38.254000+00:00,3.727,39.006,scenario 2


In [27]:
df_s2["timestamp"] = pd.to_datetime(df_s2["timestamp"])
df_s2 = df_s2.sort_values(by=["username", "timestamp"])

# 사용자별로 처리
def get_first_response_and_time(user_df):
    user_df = user_df.reset_index(drop=True)
    for i in range(len(user_df) - 1):
        curr_btn = user_df.loc[i, "button"]
        next_btn = user_df.loc[i + 1, "button"]
        
        if curr_btn == "결제 버튼" and next_btn in ["yes", "no"]:
            time_diff = (user_df.loc[i + 1, "timestamp"] - user_df.loc[i, "timestamp"]).total_seconds()
            return pd.Series([next_btn, time_diff])
    return pd.Series(["응답 없음", None])


# 사용자별 결과 계산
results_s2 = df_s2.groupby("username").apply(get_first_response_and_time).reset_index()
results_s2.columns = ["username", "결제 응답", "응답까지 걸린 시간 (초)"]

results_s2
# yes / no만 필터링
filtered = results_s2[results_s2["결제 응답"].isin(["yes", "no"])]

# 응답별 count와 평균 시간 계산
summary_s2 = (
    filtered.groupby("결제 응답")["응답까지 걸린 시간 (초)"]
    .agg(응답수="count", 평균시간="mean")
    .reset_index()
)

# 시나리오 레이블 추가
summary_s2["시나리오"] = "scenario 2"

# 결과 출력
summary_s2

C:\Users\82103\AppData\Local\Temp\ipykernel_13068\4272488158.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_s2 = df_s2.groupby("username").apply(get_first_response_and_time).reset_index()


,결제 응답,응답수,평균시간,시나리오
0,no,7,15.585143,scenario 2
1,yes,3,8.466333,scenario 2
